In [1]:
import numpy as np
import randrot as rd
import copy
import sys
sys.path.insert(0, '../')
from original import Kernels as K
import matplotlib.pyplot as plt


### Initial Definitions

In [2]:
k3 = K.ThreeBody(theta=[1., 10., 100000.])

Started compilation of theano three body kernels
Ended compilation of theano three body kernels


In [3]:
k3.theta = [.001, 1., 1.]

In [4]:
# define a function taking the positions 
# of thre atoms and creating a local configuration around the first 

def build_conf(r1, r2, r3):
    r1, r2, r3 = np.array(r1, dtype = float), np.array(r2, dtype = float), np.array(r3, dtype = float)
    riv = r2[0:3]-r1[0:3]
    rjv = r3[0:3]-r1[0:3]

    ri = np.concatenate((riv, np.array([r1[3], r2[3]])))
    rj = np.concatenate((rjv, np.array([r1[3], r3[3]])))
    conf = np.reshape(np.array([[ri],[rj]]),(1, 2, 5))
    return conf
     

### A simple configuration

In [5]:
# test that three body works correctly
s1 = 1.
s2 = 1.
s3 = 3.

In [6]:
# Define thee atoms, symmetrically positioned

r1 = [0, 0, 0, s1]
r2 = [3, 0, 0, s2]
r3 = [0, 4, 0, s3]

c1 = build_conf(r1, r2, r3)
c1

array([[[3., 0., 0., 1., 1.],
        [0., 4., 0., 1., 3.]]])

In [7]:
k3.calc_ee(c1, c1)

array([[3.]])

In [ ]:
k3.calc_ee(c1, c1)

In [8]:
# Define a random configuration

In [37]:
confR = np.random.normal(0, 1, (3,3))
confR = np.hstack((confR, np.reshape([s1, s2, s3],(3,1))))
confR_ = build_conf(confR[0], confR[1], confR[2])


In [38]:
print(confR)

[[-1.2286262   0.2954585  -1.12020994  1.        ]
 [-1.27759234  1.20948867  0.9194729   1.        ]
 [-0.13258819  1.29121865 -0.4380652   3.        ]]


In [39]:
k3.calc_ee(confR_, confR_)

array([[3.]])

In [40]:
confRP[:] = confR[:]

In [46]:
confRP[0, :] = confR[2, :]
confRP[2, :] = confR[0, :]
confRP_ = build_conf(confRP[0], confRP[1], confRP[2])

In [47]:
k3.calc_ee(confRP_, confRP_)

array([[2.]])

### Randomly sampled triplets

In [11]:
base_kernels = []

for i in np.arange(10):
    r1r = np.concatenate((np.random.normal(0, 1, (3,)),[s1]))
    r2r = np.concatenate((np.random.normal(0, 1, (3,)),[s2]))
    r3r = np.concatenate((np.random.normal(0, 1, (3,)), [s3]))
    c_ran = build_conf(r1r, r2r, r3r)
    base_kernels.append(k3.calc_ee(c_ran, c_ran)[0,0])

In [12]:
base_kernels

[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]

In [13]:
#plt.hist(base_kernels, 60)
#plt.show()

### Rndom Permutations

In [14]:
np.random.permutation(confR)

array([[ 1.26258858,  0.55844067,  0.50672498,  1.        ],
       [-0.59367035,  0.9001541 , -1.65946354,  3.        ],
       [ 0.00728949, -0.55702106,  0.37340277,  1.        ]])

In [15]:
permuted_kernels = []

for i in np.arange(10):
    rs_per = np.random.permutation(confR)
    c_per = build_conf(rs_per[0], rs_per[1], rs_per[2])
    permuted_kernels.append(k3.calc_ee(c_per, c_per)[0,0])

In [16]:
permuted_kernels

[1.0, 2.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 1.0, 2.0]

### Random Rotations

In [162]:
rotated_kernels = []

for i in np.arange(10):
    RM = rd.generate(3)
    r1r, r2r, r3r = np.array(r1[:]), np.array(r2[:]), np.array(r3[:])
    r1r[0:3], r2r[0:3], r3r[0:3] = np.array(RM.dot(r1[0:3])), np.array(RM.dot(r2[0:3])), np.array(RM.dot(r3[0:3]))
    c_rot = build_conf(r1r, r2r, r3r)
    
    rotated_kernels.append(k3.calc_ee(c_rot, c_rot)[0,0])

In [163]:
rotated_kernels

[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]